# Stats for eHRAF Scraper
The current stats are merely to 
- Clean and reorganize the dataframe 
- Find the most common OCM codes
- Find association rules (when one OCM appears this other OCM is likely to appear)
More work will be done


## Clean the Dataframe


In [ ]:
import pandas as pd                 # dataframe storing
import numpy as np
import re                           # regex for searching through strings

# put folder name here
folder = r'subjects-(sickness)_FILTERS-culture_level_samples(PSF)'


df = pd.read_excel('../Data/' + folder + '/_Altogether_Dataset.xlsx')
# Turn the string of column OCM back into a list 
df['OCM'] = df.OCM.apply(lambda x: re.sub(" ",'',x))
df['OCM'] = df.OCM.apply(lambda x: x[1:-1].split(','))

# did it work? did it output a single OCM string?
df['OCM'][0][0]

In [ ]:
# drop all rows that have a blank passage
print(f'Before: {len(df)}')
df = df.dropna(subset="Passage")
print(f'After: {len(df)}')

In [ ]:
# If you use a higher order code (750) eHRAF attempts to aquire ALL OCMs related to your input.
# select only the OCMs we originally wished to search for by inputting OCM's into a list
lst = ["750","751","752","753"]
# lst = ["751", "752"]
msk = df['OCM'].apply(lambda x: not set(x).isdisjoint(lst))
df = df.loc[msk]
len(df)

### Remove Duplicates
Currently, only passages will be removed if they contain a duplicate passage with the same OCMs. Duplicate passages with different OCMs will remain

In [ ]:
# (exploratory)  
# Find all passages which are duplicates but do not share the same document. 
# First let's explore some of the duplicates
dup1 = df["Passage"].duplicated(keep=False)  # find all duplicate passages
dup2 = df[dup1].duplicated(subset=["Passage", "DocTitle"], keep=False) #of the duplicate passages, find those that shair a passage and doc title
# rows which contain duplicate passages but not part of the same document (only top 4 shown)
print(f'Number of passages whose duplicates come from different documents: {len(df[dup1][~dup2].sort_values(by="Passage"))}')
df[dup1][~dup2].sort_values(by='Passage').head(4)

In [ ]:
# (exploratory) 
# Find passages which have duplicates but whose duplicates have different OCM numbers
df = df.copy()
df["OCM"] = df['OCM'].apply(tuple) #turn the OCM list to a tuple to allow for comparisons

# Of the passages which have duplicates, find and keep all which have the same OCM
dup3 = df[dup1].duplicated(subset=["Passage", "OCM"], keep=False)
# Show only the passages with duplicates but NOT matching OCMs
print(f'Number of passages whose duplicates do not share OCMs:  {len(df[dup1][~dup3].sort_values(by="Passage"))}')
df["OCM"] = df['OCM'].apply(list)
df[dup1][~dup3].sort_values(by="Passage").head(4)

In [ ]:
# remove all duplicated passages which share OCMs

df["OCM"] = df['OCM'].apply(tuple) #turn the OCM list to a tuple to allow for comparisons

# drop duplicates
print(f'Before {len(df)}')
df.drop_duplicates(subset=["Passage", "OCM"], keep='first', inplace=True)
print(f'After {len(df)}')

df["OCM"] = df['OCM'].apply(list) #turn the OCM back to a list

### Shave OCMs
And make an exploded OCM dataframe

In [ ]:
# Make a dataset in which each OCM have its own row by exploding (you can reset the index with .reset_index(drop=True))
df_OCM = df.explode(column='OCM').reset_index(drop=True)
# Find OCM's that do not fit the normal 100-900 OCM scheme
# NOTE 0 means the material is not relevant, I am unsure, however, why this sometimes appears with other OCM's in the same passage
# NOTE I believe 5310 and 5311 are different specifications of 531 while 1710 might be a more specific (and singlular) subset of 171? I do not believe the same for 77 and 1787
list_OCM = df_OCM['OCM'].value_counts().index.tolist()
small_OCM = [x for x in list_OCM if len(x) <3 or len(x) > 3]
small_OCM

In [ ]:
# remove and shave OCM codes
# add to the list for codes which should be removed
remove_list = ['1787','77']
print(f'starting list {len(df_OCM)}')
for i in remove_list:
    df_OCM = df_OCM[df_OCM["OCM"] != i]
# "Shave" the OCM codes that seem to have a parent (5310 and 5311 become 531).
df_OCM['OCM'] = df_OCM.OCM.apply(lambda x: x[0:3] if len(x) >= 3 else x)
print(f'Ending list {len(df_OCM)}')

In [ ]:
# Apply the removals like above to the original dataframe (this is easier than just imploding as there are duplicates which limit this)

# remove specified OCM codes
df["OCM"] = df["OCM"].apply(lambda x: [item for item in x if item not in remove_list])
# shorten the 'small_OCM' OCMs so that 5310 becomes 531
df["OCM"] = df["OCM"].apply(lambda x: [item[0:3] if item in small_OCM else item for item in x])
print(len(df))
# explantaion of above list comprehension: go through every row of the column "OCM" (via apply) 
# lambda x is an anonymous function which takes the row "x" and inputs it into the function.
# each row has its list items iterated over ( "___ for item in x") and checked if each list item is part of the small_OCM list, if so,
# return the first 3 characters, if not, return the original list item. Return everything back as a list and apply it to the dataframe


### Create Dictionary for later count comparisons

In [ ]:
# Find the number of passages for each culture
culture_set = set(df["Culture"])
culture_dict = {}
it_count = 0
for cult_i in culture_set:
    row_count = len(df.loc[df["Culture"]==cult_i])
    culture_dict[cult_i] = row_count
    it_count += row_count
print(f'Passages: {it_count}')


### Optional Exploration

In [ ]:
# (OPTIONAL)
# Quick search for OCMs regardless of culture
# NOTE, sometimes a higher order code like 750 appears without lower order codes)
lst = ["159"] #enter your OCM strings here separated by a comma
msk = df['OCM'].apply(lambda x: not set(x).isdisjoint(lst))
out = df.loc[msk]
out

In [ ]:
# Quick search for OCMs SUBINDEX BY ANOTHER COLUMN
lst = ["159"] #enter your OCM strings here separated by a comma
msk = df.loc[df["Culture"]== "Akan"]['OCM'].apply(lambda x: not set(x).isdisjoint(lst))
out = df.loc[msk.index][msk]
out

In [ ]:
# (OPTIONAL)
# There are some passages that describe previous passages but do not contain information themselves like: 
# "Notes" or "End" or "Log"
# This code cell indicates (but does not remove) how many passages are short like the ones described which 
# may disrupt our OCM stats because they contain OCMs without actually having text that refers to these OCMs
shortPass_list = []
for i in df['Passage']:
    if len(i)<=10:
        shortPass_list.append(i)
print(f'Number of passages with text with 10 of fewer characters: {len(shortPass_list)}')

## OCM Code Counting
Count every OCM within each culture. Do not count OCM's specified by the search (like if searched for 750-755, do not count these). 
<!-- - REMOVE all passages which are blank since we can't very well do lexical searches on them -->

In [ ]:
# Make a copy of df_OCM as to not interfere with other analysis
df_OCM_freq = df_OCM.copy()
# Then turn the OCM's back to an integer (for removals)
df_OCM_freq['OCM'] = df_OCM_freq.OCM.apply(lambda x: int(x))
# only keep OCMs outside our search parameters whatever those are
df_sub_ex = df_OCM_freq.loc[(df_OCM_freq["OCM"]<750) | (df_OCM_freq["OCM"]>754)]

# Overwrite and create a new dataframe for OCM counts and frequencies
df_OCM_freq = pd.DataFrame(columns=["Culture","OCM","Frequency","Proportion_of_Passages"])
for key, val in culture_dict.items():
    value_count = df_sub_ex.loc[df_sub_ex["Culture"]==key]["OCM"].value_counts()
    # duplicate the culture word and asign it to each of its rows
    cult_count = [key] * len(value_count)
    # create a culture dataframe and append it to to the 
    df_OCM_Concat = pd.DataFrame({"Culture":cult_count,"OCM":value_count.index, "Frequency":value_count.values, "Proportion_of_Passages":value_count.values/val})
    df_OCM_freq = pd.concat([df_OCM_freq, df_OCM_Concat], ignore_index=True)
df_OCM_freq = df_OCM_freq.sort_values(by = ["Culture", "Frequency"], ascending= [True, False])
df_OCM_freq

In [ ]:
print(f'OCMs per culture: {sum(df_OCM_freq["Frequency"]) / len(set(df_OCM_freq["Culture"]))}')

In [ ]:
# Save the file
df_OCM_freq.to_csv("Culture_Frequency.csv", index=False)

## Association Rules for OCMs
Using Machine Learning, we will attempt to determine the co-occurance of OCMs. For example, if the OCM code 262 is present, what is the likelihood that both 751 and 752 would be present?

In [ ]:
# Load resources
from mlxtend.preprocessing import TransactionEncoder

# We will use the apriori module to generate a dataframe that
# we can use for association rule finding
from mlxtend.frequent_patterns import apriori

# We will use the association_rules module to generate
# our association rules from the apriori output data frame
from mlxtend.frequent_patterns import association_rules





In [ ]:
#Display important columns
df_smaller = df_OCM[['Culture', 'OCM','Passage']]
df_smaller

In [ ]:
# created a grouped dataframe object by Culture and Passage 
df_group = df_smaller.groupby(by = ['Culture', 'Passage'])
df_group

In [ ]:
def make_OCM_list(x):

    '''
    Will return a list of the unique items
    in a particular grouping when used with
    the agg method as its function
    '''

    return x.unique()

In [ ]:
# Use the agg method and make_OCM_list
# to return a list of unique items for each ocm
# Note that depending on the filtering, there may be duplicate passages with different OCMs which are aggregated, 
# this method will combine them and extract the unique OCMs so it may not be a problem.
df_unique = df_group.agg(make_OCM_list)

In [ ]:
list_trans = list(df_unique['OCM'])
list_trans = list_trans[0:]
len(list_trans)

In [ ]:
te = TransactionEncoder()
encoded_itemset = te.fit(list_trans).transform(list_trans)
print(encoded_itemset.shape) # show possible transcations and number of items
te.columns_



df_encoded = pd.DataFrame(encoded_itemset, columns = te.columns_)
df_encoded.head()

In [ ]:
# Before we begin, let's do a small
# amount of cleanup.  Let's remove all
# columns (items) that have less than 1 characters since that is just blank space
# more data cleaning my be required as time continues in case errors become evident in the scraped dataset
OCM_items = list(filter(lambda x: len(x) < 1, te.columns_ ))
print("removed: ",  OCM_items)
df_encoded = df_encoded.drop(columns=OCM_items) #remove small strings as they seem not to be items
print('How many unique items are left?', len(df_encoded.columns))

In [ ]:
# Use apriori to create a dataframe with columns of support and itemset lists
# Note that if your items are large compared to your sample (you have few rows but many columns) I reccommend using 
# a higher min_support as many more combinations may have spuriously higher support. Also, you can crash the program if too many are selected
df_support = apriori(df_encoded, min_support=0.01, use_colnames=True)
df_support.sort_values('support', inplace=True, ascending = False)
df_support

## Use association_rules to find the rules

Using the dataframe generated by `apriori`, find the association rules with the greatest lift.  See the [association_rules documentation](https://rasbt.github.io/mlxtend/api_modules/mlxtend.frequent_patterns/association_rules/) for how to do this.

Sort the resulting DataFrame by lift in descending order.  A lift > 1 indicates that the items are often purchased together and that buying X will increase the purchase of Y.  A lift of < 1 indicates the items are often substituted.  That is X is substituted for Y so X and Y don't appear together often.

Examine the resulting DataFrame.  For the association rule X -> Y, X is the column `antecedents` and Y is the column `consequents`.  If sorted you can see the metrics for each rule based upon the lift.

In [ ]:
# Find the association rules
rules = association_rules(df_support, metric = 'lift', min_threshold=1.0)
# lift >1 more likely than chance X means you see Y
# lift = 1 as often as chance
# lift <1 (substitution) less likely than chance X means you see Y


In [ ]:
# Sort the rules by lift
# and examine the output
# to find what rules were
# discovered
rules.sort_values('lift', ascending=False, inplace =True)
rules

In [ ]:
# look for OCM codes within the list
lst = frozenset(["793","226"])
msk = rules['antecedents'].apply(lambda x: not set(x).isdisjoint(lst))
out = rules.loc[msk]
out